# Sentiment Analysis & Profit Data

In [1]:
#Import everything 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import csv

import matplotlib.pyplot as plt
plt.style.use('seaborn')


#starting sentiment analysis 

import nltk
from nltk.corpus import stopwords

#Lemmatization
#Basically changing word like resturants to resturant(words to word) bc really they mean the same thing 

import textblob

from textblob import Word

from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

#Sentiment Analysis

from textblob import TextBlob

# Importing the data in two DataFrames

In [2]:
movie_df = pd.read_csv('/Users/markdetiberiis/MovieMonths/movie_df.csv')

In [3]:
df_t = pd.read_csv('/Users/markdetiberiis/MovieMonths/sentiment_reviews.csv')

In [4]:
movie_df

,Unnamed: 0,Unnamed: 0.1,tconst,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,production_budget,domestic_gross,worldwide_gross,release_month,release_day,release_year
0,0,0,tt0249516,Foodfight,2012,91.0,"Action,Animation,Comedy",1.9,8248,26,45000000.0,0.0,73706.0,Dec,31,2012
1,1,1,tt0326592,The Overnight,2010,88.0,NaN,7.5,24,21,200000.0,1109808.0,1165996.0,Jun,19,2015
2,2,2,tt3844362,The Overnight,2015,79.0,"Comedy,Mystery",6.1,14828,21,200000.0,1109808.0,1165996.0,Jun,19,2015
3,3,3,tt0337692,On the Road,2012,124.0,"Adventure,Drama,Romance",6.1,37886,17,25000000.0,720828.0,9313302.0,Mar,22,2013
4,4,4,tt4339118,On the Road,2014,89.0,Drama,6.0,6,17,25000000.0,720828.0,9313302.0,Mar,22,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870,2870,2870,tt8680254,Richard III,2016,NaN,Drama,9.1,28,65,9200000.0,2684904.0,4199334.0,Dec,29,1995
2871,2871,2871,tt8824064,Heroes,2019,88.0,Documentary,7.3,7,12,400000.0,655538.0,655538.0,Oct,24,2008
2872,2872,2872,tt8976772,Push,2019,92.0,Documentary,7.3,33,70,38000000.0,31811527.0,49678401.0,Feb,6,2009
2873,2873,2873,tt9024106,Unplanned,2019,106.0,"Biography,Drama",6.3,5945,33,6000000.0,18107621.0,18107621.0,Mar,29,2019


In [5]:
df_t['primary_title'] = df_t['Unnamed: 0']

# Drop Junk Columns

In [7]:
df_t.drop(['Unnamed: 0','word_count','char_count','stopword_percentage','stopword_count','char_count','word_count'], axis = 1, inplace = True)


In [8]:
df_t

,review,review_as_string,polarity,subjectivity,primary_title
0,"['Odd adventure-comedy has lots of innuendo, c...","Odd adventure-comedy has lots of innuendo, car...",0.011111,0.416667,Foodfight
1,['This snappy role reversal may or may not app...,This snappy role reversal may or may not appea...,0.279006,0.481250,Tangled
2,"[""It's an interesting departure for Swedish ho...",It's an interesting departure for Swedish horr...,0.051822,0.535101,Shazam
3,['Margaret is less about plot mechanics than a...,Margaret is less about plot mechanics than abo...,0.111290,0.517119,Margaret
4,['Snipes sempre uma presena interessante em c...,Snipes sempre uma presena interessante em cen...,0.000000,0.000000,Unstoppable
...,...,...,...,...,...
502,"['""Serendipity"" will serve as an excellent gau...","""Serendipity"" will serve as an excellent gauge...",0.245336,0.610054,Serendipity
503,"['Outdated and schematic, this two-character m...","Outdated and schematic, this two-character mel...",0.235317,0.425198,Flawless
504,"['The subject deserved a better film.', ""Samir...","The subject deserved a better film., Samir Kar...",0.077778,0.311111,Heroes
505,[],NaN,0.000000,0.000000,Push


In [9]:
movie_df

,Unnamed: 0,Unnamed: 0.1,tconst,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,production_budget,domestic_gross,worldwide_gross,release_month,release_day,release_year
0,0,0,tt0249516,Foodfight,2012,91.0,"Action,Animation,Comedy",1.9,8248,26,45000000.0,0.0,73706.0,Dec,31,2012
1,1,1,tt0326592,The Overnight,2010,88.0,NaN,7.5,24,21,200000.0,1109808.0,1165996.0,Jun,19,2015
2,2,2,tt3844362,The Overnight,2015,79.0,"Comedy,Mystery",6.1,14828,21,200000.0,1109808.0,1165996.0,Jun,19,2015
3,3,3,tt0337692,On the Road,2012,124.0,"Adventure,Drama,Romance",6.1,37886,17,25000000.0,720828.0,9313302.0,Mar,22,2013
4,4,4,tt4339118,On the Road,2014,89.0,Drama,6.0,6,17,25000000.0,720828.0,9313302.0,Mar,22,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870,2870,2870,tt8680254,Richard III,2016,NaN,Drama,9.1,28,65,9200000.0,2684904.0,4199334.0,Dec,29,1995
2871,2871,2871,tt8824064,Heroes,2019,88.0,Documentary,7.3,7,12,400000.0,655538.0,655538.0,Oct,24,2008
2872,2872,2872,tt8976772,Push,2019,92.0,Documentary,7.3,33,70,38000000.0,31811527.0,49678401.0,Feb,6,2009
2873,2873,2873,tt9024106,Unplanned,2019,106.0,"Biography,Drama",6.3,5945,33,6000000.0,18107621.0,18107621.0,Mar,29,2019


# Combine Both DataFrames

In [11]:
sent_profit_df = pd.merge(movie_df, df_t)
sent_profit_df.head(10)

,Unnamed: 0,Unnamed: 0.1,tconst,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,production_budget,domestic_gross,worldwide_gross,release_month,release_day,release_year,review,review_as_string,polarity,subjectivity
0,0,0,tt0249516,Foodfight,2012,91.0,"Action,Animation,Comedy",1.9,8248,26,45000000.0,0.0,73706.0,Dec,31,2012,"['Odd adventure-comedy has lots of innuendo, c...","Odd adventure-comedy has lots of innuendo, car...",0.011111,0.416667
1,12,12,tt0398286,Tangled,2010,100.0,"Adventure,Animation,Comedy",7.8,366366,15,260000000.0,200821936.0,586477240.0,Nov,24,2010,['This snappy role reversal may or may not app...,This snappy role reversal may or may not appea...,0.279006,0.481250
2,26,26,tt0448115,Shazam,2019,132.0,"Action,Adventure,Comedy",7.4,109051,97,85000000.0,139606856.0,362899733.0,Apr,5,2019,"[""It's an interesting departure for Swedish ho...",It's an interesting departure for Swedish horr...,0.051822,0.535101
3,39,39,tt0466893,Margaret,2011,150.0,Drama,6.5,14708,27,14000000.0,47185.0,623292.0,Sep,30,2011,['Margaret is less about plot mechanics than a...,Margaret is less about plot mechanics than abo...,0.111290,0.517119
4,45,45,tt0477080,Unstoppable,2010,98.0,"Action,Thriller",6.8,173019,19,95000000.0,81562942.0,165720921.0,Nov,12,2010,['Snipes sempre uma presena interessante em c...,Snipes sempre uma presena interessante em cen...,0.000000,0.000000
5,46,46,tt3070502,Unstoppable,2013,65.0,Documentary,4.3,551,19,95000000.0,81562942.0,165720921.0,Nov,12,2010,['Snipes sempre uma presena interessante em c...,Snipes sempre uma presena interessante em cen...,0.000000,0.000000
6,47,47,tt9225192,Unstoppable,2018,116.0,"Action,Crime",6.5,576,19,95000000.0,81562942.0,165720921.0,Nov,12,2010,['Snipes sempre uma presena interessante em c...,Snipes sempre uma presena interessante em cen...,0.000000,0.000000
7,48,48,tt9906218,Unstoppable,2019,84.0,Documentary,8.1,8,19,95000000.0,81562942.0,165720921.0,Nov,12,2010,['Snipes sempre uma presena interessante em c...,Snipes sempre uma presena interessante em cen...,0.000000,0.000000
8,58,58,tt0490215,Silence,2016,161.0,"Adventure,Drama,History",7.2,85242,42,46500000.0,7100177.0,23726626.0,Dec,23,2016,[],NaN,0.000000,0.000000
9,59,59,tt1731755,Silence,2011,110.0,"Drama,War",6.7,373,42,46500000.0,7100177.0,23726626.0,Dec,23,2016,[],NaN,0.000000,0.000000


In [12]:
#Drop junk columns 

sent_profit_df.drop(['review','review_as_string'], axis = 1, inplace = True)


In [13]:
#Just for safe keeping 

sent_profit_df.to_csv('/Users/markdetiberiis/MovieMonths/sent&profit.csv')
